1.feature with same err or not
2.err value

In [20]:
import jupyter_Importer
from MLModels_FeatureImportance import featureImportance_linearRegression
from MLModels_FeatureImportance import featureImportance_permutation
from MLModels_FeatureImportance import featureImportance_DTree

import pandas as pd
import numpy as np
import math
from sklearn import preprocessing

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

In [21]:
dataset = pd.read_csv("Occupancy.csv", names = ['x' + str(i) for i in range(1, 7)] + ['y'], sep = ",")
dataset.head()

,x1,x2,x3,x4,x5,x6,y
0,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
1,2015-02-02 14:19:00,23.7,26.272,585.2,749.2,0.0047641630241641,1
2,2015-02-02 14:19:59,23.718,26.29,578.4,760.4,0.0047726609921251,1
3,2015-02-02 14:21:00,23.73,26.23,572.666666666667,769.666666666667,0.0047651525524654,1
4,2015-02-02 14:22:00,23.7225,26.125,493.75,774.75,0.0047437733559968,1


In [22]:
dataset_light_co2 = dataset.iloc[1:,3:5]
dataset_light_co2['y'] = dataset.iloc[1:,6:7]
dataset_light_co2.columns = ['x1','x2','y']
dataset_light_co2['x1'] = dataset_light_co2['x1'].apply(lambda x : float(x))
dataset_light_co2['x2'] = dataset_light_co2['x2'].apply(lambda x : float(x))
dataset_light_co2['y'] = dataset_light_co2['y'].apply(lambda x : int(x))
dataset_light_co2

,x1,x2,y
1,585.200000,749.200000,1
2,578.400000,760.400000,1
3,572.666667,769.666667,1
4,493.750000,774.750000,1
5,488.600000,779.000000,1
...,...,...,...
20556,429.750000,1505.250000,1
20557,423.500000,1514.500000,1
20558,423.500000,1521.500000,1
20559,418.750000,1632.000000,1


In [23]:
subset = dataset_light_co2.sample(frac=0.2,replace=False,random_state=43)
dataset_light_co2 = dataset_light_co2.append(subset)
dataset_light_co2 = dataset_light_co2.drop_duplicates(subset=['x1','x2','y'],keep=False)

subsample_x = subset.iloc[:,:-1].values
subsample_y = subset.iloc[:,-1].values

print(dataset_light_co2.info(),subset.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6757 entries, 1 to 20560
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      6757 non-null   float64
 1   x2      6757 non-null   float64
 2   y       6757 non-null   int64  
dtypes: float64(2), int64(1)
memory usage: 211.2 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4112 entries, 8855 to 4674
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      4112 non-null   float64
 1   x2      4112 non-null   float64
 2   y       4112 non-null   int64  
dtypes: float64(2), int64(1)
memory usage: 128.5 KB
None None


In [24]:
from sklearn import preprocessing
subsample_x = pd.DataFrame(subsample_x)
minmax = preprocessing.MinMaxScaler()
data_minmax = minmax.fit_transform(subsample_x)

In [25]:
#preprocess the data
dataset_light_co2_nml = dataset_light_co2.iloc[:,:-1]
minmax = preprocessing.MinMaxScaler()
dataset_light_co2_nml = minmax.fit_transform(dataset_light_co2_nml)
#dataset_light_co2_nml = preprocessing.scale(dataset_light_co2_nml)
print(dataset_light_co2_nml.shape,dataset_light_co2.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6757 entries, 1 to 20560
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      6757 non-null   float64
 1   x2      6757 non-null   float64
 2   y       6757 non-null   int64  
dtypes: float64(2), int64(1)
memory usage: 211.2 KB
(6757, 2) None


In [26]:
dataset_light_co2_nml = pd.DataFrame(dataset_light_co2_nml)
dataset_light_co2_nml.columns = ['x1','x2']
#dataset_light_co2_nml['y'] = dataset_light_co2['y']
dataset_light_co2_nml.insert(2,'y',dataset_light_co2['y'].values)

x1_min = -1.333440
x1_max = 5.659035
x2_min = -1.260885
x2_max = 3.575273
print(dataset_light_co2_nml.max(),dataset_light_co2_nml.min())
print(dataset_light_co2_nml.info(),dataset_light_co2.info())

x1    1.0
x2    1.0
y     1.0
dtype: float64 x1    0.0
x2    0.0
y     0.0
dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6757 entries, 0 to 6756
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      6757 non-null   float64
 1   x2      6757 non-null   float64
 2   y       6757 non-null   int64  
dtypes: float64(2), int64(1)
memory usage: 158.5 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6757 entries, 1 to 20560
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      6757 non-null   float64
 1   x2      6757 non-null   float64
 2   y       6757 non-null   int64  
dtypes: float64(2), int64(1)
memory usage: 211.2 KB
None None


In [27]:
def df_2feature_be_generates(BEi,BEj) :#i for brute force or not,n for negative importance
    temp = dataset_light_co2.copy()
    df_2feature_be = pd.DataFrame.to_numpy(dataset_light_co2)
    
    e1 = temp['x1'].mean()*BEi
    e2 = temp['x2'].mean()*BEj
    
    
    for idx,cell in temp['x1'].iteritems():
        temp['x1'][idx] += e1
    temp = temp.to_numpy()
    df_2feature_be = np.vstack((df_2feature_be,temp))
    #print(df_2feature_be.shape)
    temp = dataset_light_co2.copy()

    for idx,cell in temp['x1'].iteritems():
        temp['x1'][idx] = temp['x1'][idx] - e1
    temp = temp.to_numpy()
    df_2feature_be = np.vstack((df_2feature_be,temp))
    #print(df_2feature_be.shape)
    temp = dataset_light_co2.copy()

    for idx,cell in temp['x2'].iteritems():
        temp['x2'][idx] += e2
    temp = temp.to_numpy()
    df_2feature_be = np.vstack((df_2feature_be,temp))
    #print(df_2feature_be.shape)
    temp = dataset_light_co2.copy()

    for idx,cell in temp['x2'].iteritems():
        temp['x2'][idx] = temp['x2'][idx] - e2
    temp = temp.to_numpy()
    df_2feature_be = np.vstack((df_2feature_be,temp))
    #print(df_2feature_be.shape)
    temp = dataset_light_co2.copy()

    df_2feature_be = pd.DataFrame(df_2feature_be)
    #print(df_2feature_be)
    return df_2feature_be

In [30]:
err_rateA = [0,0.1,0.2,0.3]
err_rateB = [0,0.1,0.2,0.3]

for i in err_rateA:
    for j in err_rateB:
        df_2feature_be = df_2feature_be_generates(i,j)
        x_be = df_2feature_be.iloc[:, :-1].values
        y_be = df_2feature_be.iloc[:,-1].values
        X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
        clf_BE = SVC(kernel='linear',C=1,gamma='auto')
        clf_BE.fit(X_train, y_train)
        y_BE_predict = clf_BE.predict(X_test)
        result_BE_train = clf_BE.score(X_train, y_train)
        result_BE_test = clf_BE.score(X_test, y_test)
        subset_raw = clf_BE.score(subsample_x, subsample_y)
        print('BESVM Accuracy (on training) = with x1 bounded error ('+str(i)+'),x2 with error ('+str(j)+') : '+str(result_BE_train))
        print('BESVM Accuracy (on testing) = with x1 bounded error ('+str(i)+'),x2 with error ('+str(j)+') : '+str(result_BE_train))
        print('BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))



d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-

BESVM Accuracy (on training) = with x1 bounded error (0),x2 with error (0) : 0.971733017611366
BESVM Accuracy (on testing) = with x1 bounded error (0),x2 with error (0) : 0.971733017611366
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9868677042801557


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-

BESVM Accuracy (on training) = with x1 bounded error (0),x2 with error (0.1) : 0.9733979576735238
BESVM Accuracy (on testing) = with x1 bounded error (0),x2 with error (0.1) : 0.9733979576735238
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9875972762645915


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-

...

d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-

BESVM Accuracy (on training) = with x1 bounded error (0.3),x2 with error (0.2) : 0.9530856889151991
BESVM Accuracy (on testing) = with x1 bounded error (0.3),x2 with error (0.2) : 0.9530856889151991
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9854085603112841


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-

BESVM Accuracy (on training) = with x1 bounded error (0.3),x2 with error (0.3) : 0.9510507621725618
BESVM Accuracy (on testing) = with x1 bounded error (0.3),x2 with error (0.3) : 0.9510507621725618
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9858949416342413


: 